In [42]:
# импортируем библиотеки
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from random import randint

import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 

pd.options.display.max_columns = None

# Цель проекта

вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:

-   В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
-   Строят модель для предсказания объёма запасов в новых скважинах;
-   Выбирают скважины с самыми высокими оценками значений;
-   Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой _Bootstrap._




Цель проекта: определить один из трёх регионов, гдед обыча принесет наибольшую прибыль

### Загрузите и подготовьте данные. Поясните порядок действий.

In [43]:
#импортируем данные
sources = {}
filelist=['geo_data_2','geo_data_0','geo_data_1']

for f in filelist:
    pth12 = '/datasets/'+f+'.csv'
    pth11 =f+'.csv'

    if os.path.exists(pth12):
        sources[f] = pd.read_csv(pth12)
    elif os.path.exists(pth11):
        sources[f] = pd.read_csv(pth11)
    else:
        print('Something is wrong')

In [44]:
display(sources['geo_data_2'].info())
display(sources['geo_data_2'].sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
37989,CO1Hp,3.873058,0.060201,-0.742090,137.340943
30283,5zcTw,-0.633226,-0.203539,7.442949,110.260765
29444,8gwpX,0.704070,0.306076,4.777151,142.437417
91583,SrtEL,-0.070279,2.663167,4.559973,79.759576
93840,v6hCs,1.469478,-0.282742,3.405149,142.420077


# Обучите и проверьте модель для каждого региона:

подготовим словарь для результатов предсказаний, с ключами по названиям месторождений:

In [45]:
results = {}

##### Инициализация модели линейной регрессии

In [46]:
def get_predictions(df,results,data_name):
    features_in = df.drop(['product','id'], axis=1)
    target_in = df['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
       features_in, target_in, test_size=0.25, random_state=12345)
    
    scaler = StandardScaler()
    scaler.fit(features_train) 

    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid) 
    features_in = scaler.transform(features_in) 
    
    ##### Обучите модель и сделайте предсказания на валидационной выборке.
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions = model.predict(features_in)

    
    score = model.score(features_valid, target_valid)
    print('model.score',score)

    ##### Сохраните предсказания и правильные ответы на валидационной выборке.     
    results[data_name] = pd.merge(pd.DataFrame(predictions),df['id'],left_index=True, right_index=True)    

    ##### Напечатайте на экране средний запас предсказанного сырья и _RMSE_ модели.
    from sklearn.metrics import mean_squared_error
    print('средний запас предсказанного сырья :',predictions.mean())

    mse = mean_squared_error(target_in, predictions)
    rmse = mse ** 0.5
    print('_RMSE_ модели:', rmse)

    return results

Обучим модели и получим предсказания по каждому месторождению:

In [47]:
for k in sources.keys():
    print('\n','Месторождение:',k)
    results = get_predictions(sources[k],results,k)



 Месторождение: geo_data_2
model.score 0.20524758386040443
средний запас предсказанного сырья : 95.02020328978764
_RMSE_ модели: 40.055624818752136

 Месторождение: geo_data_0
model.score 0.27994321524487786
средний запас предсказанного сырья : 92.62849276088778
_RMSE_ модели: 37.69240596744535

 Месторождение: geo_data_1
model.score 0.9996233978805127
средний запас предсказанного сырья : 68.82635271777151
_RMSE_ модели: 0.8903802650297569


переименуем столбцы

In [48]:
for k in results.keys():
    results[k].rename({0:'val'},axis='columns', inplace=True)

# 3.  Подготовьтесь к расчёту прибыли:

####    3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.

In [49]:
#При разведке региона исследуют 500 точек
POINTS_ALL = 500 

#исследуют точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
POINTS_SELECTED= 200

#Бюджет на разработку скважин в регионе — 10 млрд рублей.
BUDGET_ALL = 10000000000

#При нынешних ценах один баррель сырья приносит 450 рублей дохода. 
#Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
REVENUE_BARELL_PRODUCT = 450000

#После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%.
MAX_RISC_LOSS = 2.5

BOOTSTRAP_ITER_NUM = 1000  

#### 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [50]:
#бюджет на одну скважину
budget_point = BUDGET_ALL / POINTS_SELECTED
print('бюджет на одну скважину budget_point:',budget_point)

#достаточный объём сырья для безубыточной разработки новой скважины
barrel_needed_point = budget_point / REVENUE_BARELL_PRODUCT 
print('достаточный объём сырья для безубыточной разработки новой скважины barrel_needed_point:',barrel_needed_point)

бюджет на одну скважину budget_point: 50000000.0
достаточный объём сырья для безубыточной разработки новой скважины barrel_needed_point: 111.11111111111111


если рассматривать данные в целом, получим средний прогноз по добыче на скважину по всем месторождениям:

In [51]:
for k in sources.keys():
    mean_place = sources[k]['product'].mean()
    print('месторождение:',k,' имеет средние запасы на скважину: ',mean_place)

месторождение: geo_data_2  имеет средние запасы на скважину:  95.00000000000004
месторождение: geo_data_0  имеет средние запасы на скважину:  92.50000000000001
месторождение: geo_data_1  имеет средние запасы на скважину:  68.82500000000002


     3.3. Напишите выводы по этапу подготовки расчёта прибыли.

Вывод: расчет показал, что нам необходимо, что скважина добывала не меньше следующего количества тысяч баррелей, чтобы быть безубыточной 111. Однако, ни одно месторождение не может этого гарантировать если рассматривать данные в целом. Попробуем сделать выборку из ограниченного количества скважен, но с необходимым(безубыточным) количеством нефти.

# 4.  Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

На каждой итерации бутстрапа мы имитируем реальный процесс отбора скважин, как если бы мы это делали в боевых условиях и затем смотрим насколько хорошо мы отобрали скважины рассчитывая прибыль по реальным значениям (таргетам). То есть: выбираем 500 случайно, делаем предсказания и по ним отбираем 200 лучших, затем для этих 200 считаем прибыль уже по таргету.

In [74]:
def revenue_well_analit(well_500, oilfield, disp=True):
    # отбираем 200 лучших
    # 4.1. Выберите скважины с максимальными значениями предсказаний.
    well_200 = well_500.sort_values(by = 'val',ascending=False).head(POINTS_SELECTED)
    
    # 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    summary_oil_200 = well_200['val'].sum()
    if disp: print('месторождение:',oilfield,' имеет запасы: ',summary_oil_200)
    
    #для расчета прибыли вычитаем бездоходные барелли 
    well_200['product'] = well_200['product'] - barrel_needed_point
    

    # для этих 200 считаем прибыль уже по таргету
    # 4.3. Рассчитайте прибыль для полученного объёма сырья.
    well_200['revenue'] = REVENUE_BARELL_PRODUCT * well_200['product']
    
    summary_revenue_200 = well_200['revenue'].sum()
    
    if disp: print('прибыль для полученного объёма сырья', summary_revenue_200)
    
    return summary_oil_200, summary_revenue_200

In [75]:
def distrib_revenue_wells(sources,results, disp=True):
    state = np.random.RandomState(12345)
     
    revenues = {}
    
    
    for oilfield in sources.keys():
        revenue_oilfield = []
        # "склеим" вместе датафреймы исходных данных и результата для удобства
        src_res = pd.merge(sources[oilfield],results[oilfield],right_index = True,left_index = True)

        for i in range(BOOTSTRAP_ITER_NUM):
            
            well_500 = src_res.sample(n=POINTS_ALL, replace=True, random_state=state)

            summary_oil_200, summary_revenue_200 = revenue_well_analit(well_500, oilfield, disp)
            
            revenue_oilfield.append(summary_revenue_200)
            
        revenues[oilfield] = revenue_oilfield
            
    return revenues
            

In [76]:
from sklearn.metrics import mean_squared_error
rmse1 = mean_squared_error(src_res['product'], src_res['val']) ** 0.5
rmse2 = mean_squared_error(sources['geo_data_2']['product'], results['geo_data_2']['val']) ** 0.5
print(rmse1,' == ',rmse2)


40.055624818752136  ==  40.055624818752136


# 5.  Посчитайте риски и прибыль для каждого региона:

Применим технику _Bootstrap_ с 1000 выборок, чтобы найти распределение прибыли.

Используем найденные максимумальные 200 самых насыщенных скважины в каждом месторождении. 

Их прибыль будет произведением объема добычи на цену (тыс. бареллей).

In [77]:
revenues = distrib_revenue_wells(sources,results, disp=False)

In [78]:
revenues = pd.DataFrame(revenues)

In [80]:
#нужно посчитать 3 статистики:

#1. Среднюю прибыль
for oilfield in sources.keys():
#     mean_rev_oilfield = (revenues[oilfield]-barrel_needed_point * REVENUE_BARELL_PRODUCT).mean()
    mean_rev_oilfield = (revenues[oilfield]).mean()
    print('Среднюю прибыль ',oilfield,' ',mean_rev_oilfield)

#2. Интервал с помощью метода квантилей
for oilfield in sources.keys():
    values = revenues[oilfield] 

    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print( '\n распределение прибыли для месторождения:',oilfield)
    print('lower:',lower)
    print('upper:',upper)

#3. Риск убытков – долю случаев, в которых прибыль отрицательна             
for oilfield in sources.keys():
    revenues_loss = pd.DataFrame(revenues[oilfield])
    revenues_loss['rev_loss'] = revenues_loss - barrel_needed_point * REVENUE_BARELL_PRODUCT 
    loss_risc = (revenues_loss['rev_loss']<0).mean()
    print('вероятность убытка ',oilfield,':', loss_risc)
        


Среднюю прибыль  geo_data_2   373939021.9898514
Среднюю прибыль  geo_data_0   416085060.41390604
Среднюю прибыль  geo_data_1   449074619.28490525

 распределение прибыли для месторождения: geo_data_2
lower: -181948188.44712105
upper: 896299408.1008723

 распределение прибыли для месторождения: geo_data_0
lower: -99884976.31121099
upper: 917289304.2100798

 распределение прибыли для месторождения: geo_data_1
lower: 57453336.00421656
upper: 846184540.2423898
вероятность убытка  geo_data_2 : 0.126
вероятность убытка  geo_data_0 : 0.071
вероятность убытка  geo_data_1 : 0.024


# Вывод

Вывод: С высокой долей вероятности самый прибыльный регион это geo_data_2. Так как в этом регионе самая большая прогнозируемая прибыль. Однако в регионе geo_data_1 риски меньше остальных.